Using edited code from [Michael Wy Ong](https://medium.com/@michael.wy.ong/web-scrape-geospatial-data-analyse-singapores-property-price-part-i-276caba320b) with some tweakings

In [49]:
import pandas as pd
import numpy as np
import requests
import json

In [50]:
## Function for getting postal code, geo coordinates of addresses

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

In [51]:
## Function for getting closest distance of each location from a list of amenities location

from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [52]:
def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results


In [53]:
# price1999 = pd.read_csv('Data/resale-flat-prices-based-on-approval-date-1990-1999.csv')
# price2012 = pd.read_csv('Data/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
# price2014 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
# price2016 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
# price2017 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

price1 = pd.read_csv('Data/Data_2012_2014.csv')
price2 = pd.read_csv('Data/Data_2015_2016.csv')
price3 = pd.read_csv('Data/Data_2017_onwards.csv')


# prices = pd.concat([price1999, price2012, price2014], sort=False)
prices = pd.concat([price1, price2, price3], axis=0, ignore_index=True, sort=False)

In [54]:
prices = prices[['block', 'street_name']]
prices['address'] = prices['block'] + ' ' + prices['street_name']
all_address = list(prices['address'])
unique_address = list(set(all_address))

print('Unique addresses:', len(unique_address))

Unique addresses: 9563


In [55]:
unique_address[:10]

['684B EDGEDALE PLAINS',
 '630 PASIR RIS DR 3',
 '118 MARSILING RISE',
 '42 JLN BAHAGIA',
 '930 YISHUN CTRL 1',
 '654C JURONG WEST ST 61',
 '311 SHUNFU RD',
 '17 CANTONMENT CL',
 '565 PASIR RIS ST 51',
 '478 SEMBAWANG DR']

In [56]:
find_postal(unique_address, 'Data/flat_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=684B EDGEDALE PLAINS
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=630 PASIR RIS DR 3
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=118 MARSILING RISE
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=42 JLN BAHAGIA
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=930 YISHUN CTRL 1
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=654C JURONG WEST ST 61
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=311 SHUNFU RD
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=17 CANTONMENT CL
8 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDe

In [57]:
flat_coord = pd.read_csv('Data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

,address,LATITUDE,LONGITUDE
0,684B EDGEDALE PLAINS,1.403027,103.915006
1,630 PASIR RIS DR 3,1.378382,103.941097
2,118 MARSILING RISE,1.438484,103.780381
3,42 JLN BAHAGIA,1.327150,103.856962
4,930 YISHUN CTRL 1,1.426915,103.837351


## Supermarkets

In [58]:
supermarket = pd.read_csv('Data/list-of-supermarket-licences.csv')
supermarket.head()

,licence_num,licensee_name,building_name,block_house_num,level_num,unit_num,street_name,postal_code
0,S02185J000,COLD STORAGE SINGAPORE (1983) PTE LTD,na,982,1,1,BUANGKOK CRESCENT,530982
1,S97166P000,COLD STORAGE SINGAPORE (1983) PTE LTD,HOUGANG GREEN SHOPPING MALL,21,1,52,HOUGANG STREET 51,538719
2,CE15B61K000,U STARS SUPERMARKET PTE. LTD.,na,330,1,1,ANCHORVALE STREET,540330
3,S02029J000,NTUC Fairprice Co-operative Ltd,HOUGANG POINT,1,2,1,HOUGANG STREET 91,538692
4,CE04561V000,SHENG SIONG SUPERMARKET PTE LTD,na,19,1,42401,SERANGOON NORTH AVENUE 5,554913


In [59]:
supermerket_address = list(supermarket['postal_code'])
unique_supermarket_address = list(set(supermerket_address))

print('Unique addresses:', len(unique_supermarket_address))

Unique addresses: 445


In [60]:
# find_postal(unique_supermarket_address, 'Data/supermarket_coordinates')

In [61]:
supermarket_coord = pd.read_csv('Data/supermarket_coordinates.csv')
supermarket_coord.drop_duplicates(subset=['address'], inplace=True)
supermarket_coord = supermarket_coord[['SEARCHVAL','LATITUDE','LONGITUDE']]
supermarket_coord.head()

,SEARCHVAL,LATITUDE,LONGITUDE
0,AYER RAJAH MARKET,1.311907,103.759128
2,MAGHAIN ABOTH SYNAGOGUE,1.298247,103.850683
3,WOODLANDS DORMITORY,1.454724,103.804585
4,UOB I12 KATONG AUTOLOBBY,1.305223,103.905044
8,PUNGGOL SPECTRA,1.400531,103.910537


In [62]:
nearest_supermarket = find_nearest(flat_coord, supermarket_coord)
flat_supermarket = pd.DataFrame.from_dict(nearest_supermarket).T
flat_supermarket = flat_supermarket.rename(columns={0: 'flat', 1: 'supermarket', 2: 'supermarket_dist', 3: 'num_supermarket_2km'}).reset_index().drop(['index','supermarket'], axis=1)
flat_supermarket.head()

,flat,supermarket_dist,num_supermarket_2km
0,684B EDGEDALE PLAINS,0.411221,12
1,630 PASIR RIS DR 3,0.101067,6
2,118 MARSILING RISE,0.805409,8
3,42 JLN BAHAGIA,0.283154,22
4,930 YISHUN CTRL 1,0.069918,15


In [63]:
flat_supermarket.to_csv('Data/flat_supermarket.csv', index=False)

## Schools

In [64]:
school = pd.read_csv('Data/school_names.csv', encoding='cp1252')
school.head(10)

,school,area
0,Admiralty Primary School,Woodlands
1,Ahmad Ibrahim Primary School,Yishun
2,Ai Tong School,Bishan
3,Alexandra Primary School,Bukit Merah
4,Anchor Green Primary School,Sengkang
5,Anderson Primary School,Ang Mo Kio
6,Anglo-Chinese School (Junior),Central
7,Anglo-Chinese School (Primary),Bukit Timah
8,Angsana Primary School,Tampines
9,Ang Mo Kio Primary School,Ang Mo Kio


In [65]:
school_name = list(school['school'])
unique_school_name = list(set(school_name))

print('Unique addresses:', len(unique_school_name))

Unique addresses: 191


In [66]:
find_postal(unique_school_name, 'Data/school_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Methodist Girls' School (Primary)
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Beacon Primary School
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Juying Primary School
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=River Valley Primary School
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Coral Primary School
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Frontier Primary School
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=St. Hilda's Primary School
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=West Grove Primary School
8 https://de

In [67]:
school_coord = pd.read_csv('Data/school_coordinates.csv')
school_coord = school_coord[['address','LATITUDE','LONGITUDE']]
school_coord.head()

,address,LATITUDE,LONGITUDE
0,Methodist Girls' School (Primary),1.349473,103.884971
1,Beacon Primary School,1.383949,103.773632
2,River Valley Primary School,1.294183,103.836019
3,River Valley Primary School,1.294223,103.835815
4,Frontier Primary School,1.336643,103.699683


In [68]:
nearest_school = find_nearest(flat_coord, school_coord)
flat_school = pd.DataFrame.from_dict(nearest_school).T
flat_school = flat_school.rename(columns={0: 'flat', 1: 'school', 2: 'school_dist', 3: 'num_school_2km'}).reset_index().drop('index', axis=1)
flat_school.head()

,flat,school,school_dist,num_school_2km
0,684B EDGEDALE PLAINS,Horizon Primary School,0.423743,16
1,630 PASIR RIS DR 3,Park View Primary School,0.205457,6
2,118 MARSILING RISE,Fuchun Primary School,0.901488,14
3,42 JLN BAHAGIA,Hong Wen School,0.600572,14
4,930 YISHUN CTRL 1,Xishan Primary School,0.726366,14


In [69]:
flat_school.to_csv('Data/flat_school.csv', index=False)

## Hawker and Markets

In [70]:
# hawker = pd.read_csv('Data/list-of-government-markets-hawker-centres.csv')
# hawker.head(10)

In [71]:
# hawker_name = list(hawker['name_of_centre'])
# unique_hawker_name = list(set(hawker_name))

# print('Unique addresses:', len(unique_hawker_name))

In [72]:
# find_postal(unique_hawker_name, 'Data/hawker_coordinates')

In [73]:
# hawker_coord = pd.read_csv('Data/hawker_coordinates.csv')
# hawker_coord = hawker_coord[['address','LATITUDE','LONGITUDE']]
# hawker_coord.head()

In [74]:
# nearest_hawker = find_nearest(flat_coord, hawker_coord)
# flat_hawker = pd.DataFrame.from_dict(nearest_hawker).T
# flat_hawker = flat_hawker.rename(columns={0: 'flat', 1: 'hawker', 2: 'hawker_dist', 3: 'num_hawker_2km'}).reset_index().drop('index', axis=1)
# flat_hawker.head()

In [75]:
# flat_hawker.to_csv('Data/flat_hawker.csv', index=False)

## Shopping Malls

In [76]:
# shop = pd.read_csv('Data/shoppingmalls.csv', encoding='cp1252')
# shop.head()

In [77]:
# shop_name = list(shop['name'])
# unique_shop_name = list(set(shop_name))

# print('Unique addresses:', len(unique_shop_name))

In [78]:
# find_postal(unique_shop_name, 'Data/shoppingmall_coordinates')

In [79]:
# shop_coord = pd.read_csv('Data/shoppingmall_coordinates_clean.csv')
# shop_coord.drop_duplicates(subset=['address'], inplace=True)
# shop_coord = shop_coord[['address','LATITUDE','LONGITUDE']]
# shop_coord.head()

In [80]:
# nearest_mall = find_nearest(flat_coord, shop_coord)
# flat_mall = pd.DataFrame.from_dict(nearest_mall).T
# flat_mall = flat_mall.rename(columns={0: 'flat', 1: 'mall', 2: 'mall_dist', 3: 'num_mall_2km'}).reset_index().drop('index', axis=1)
# flat_mall.head()

In [81]:
# flat_mall.to_csv('Data/flat_mall.csv', index=False)

## Parks

In [82]:
# park_coord = pd.read_csv('Data/parks_coordinates_clean.csv')
# park_coord.reset_index(inplace=True)
# park_coord = park_coord[['index','Y','X']]
# park_coord.head()

In [83]:
# nearest_park = find_nearest(flat_coord, park_coord)
# flat_park = pd.DataFrame.from_dict(nearest_park).T
# flat_park = flat_park.rename(columns={0: 'flat', 1: 'park', 2: 'park_dist', 3: 'num_park_2km'}).reset_index().drop(['index','park'], axis=1)
# flat_park.head()

In [84]:
# flat_park.to_csv('Data/flat_park.csv', index=False)

## MRT

In [85]:
mrt_coord = pd.read_csv('Data/MRT_coordinates.csv')
mrt_coord = mrt_coord[['STN_NAME','Latitude','Longitude']]
mrt_coord.head()

,STN_NAME,Latitude,Longitude
0,ADMIRALTY MRT STATION,1.440585,103.800998
1,ALJUNIED MRT STATION,1.316433,103.882893
2,ANG MO KIO MRT STATION,1.369933,103.849553
3,BAKAU LRT STATION,1.388093,103.905418
4,BANGKIT LRT STATION,1.380018,103.772667


In [86]:
nearest_mrt = find_nearest(flat_coord, mrt_coord)
flat_mrt = pd.DataFrame.from_dict(nearest_mrt).T
flat_mrt = flat_mrt.rename(columns={0: 'flat', 1: 'mrt', 2: 'mrt_dist', 3: 'num_mrt_2km'}).reset_index().drop('index', axis=1)
flat_mrt.head()

,flat,mrt,mrt_dist,num_mrt_2km
0,684B EDGEDALE PLAINS,OASIS LRT STATION,0.268944,18
1,630 PASIR RIS DR 3,PASIR RIS MRT STATION,1.084685,1
2,118 MARSILING RISE,WOODLANDS MRT STATION,0.703757,5
3,42 JLN BAHAGIA,BOON KENG MRT STATION,1.005326,8
4,930 YISHUN CTRL 1,YISHUN MRT STATION,0.382512,3


In [87]:
flat_mrt.to_csv('Data/flat_mrt.csv', index=False)

In [88]:
# flat_coord = pd.read_csv('Data/flat_coordinates_clean.csv')
flat_coord = pd.read_csv('Data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

,address,LATITUDE,LONGITUDE
0,684B EDGEDALE PLAINS,1.403027,103.915006
1,630 PASIR RIS DR 3,1.378382,103.941097
2,118 MARSILING RISE,1.438484,103.780381
3,42 JLN BAHAGIA,1.327150,103.856962
4,930 YISHUN CTRL 1,1.426915,103.837351


## Merge All

In [89]:
## Merge all

# flat_amenities = flat_school.merge(flat_hawker, on='flat', how='outer')
# flat_amenities = flat_amenities.merge(flat_park, on='flat', how='outer')
# flat_amenities = flat_amenities.merge(flat_mall, on='flat', how='outer')
flat_amenities = flat_school.merge(flat_mrt, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_supermarket, on='flat', how='outer')

flat_amenities.head()

,flat,school,school_dist,num_school_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km
0,684B EDGEDALE PLAINS,Horizon Primary School,0.423743,16,OASIS LRT STATION,0.268944,18,0.411221,12
1,630 PASIR RIS DR 3,Park View Primary School,0.205457,6,PASIR RIS MRT STATION,1.084685,1,0.101067,6
2,118 MARSILING RISE,Fuchun Primary School,0.901488,14,WOODLANDS MRT STATION,0.703757,5,0.805409,8
3,42 JLN BAHAGIA,Hong Wen School,0.600572,14,BOON KENG MRT STATION,1.005326,8,0.283154,22
4,930 YISHUN CTRL 1,Xishan Primary School,0.726366,14,YISHUN MRT STATION,0.382512,3,0.069918,15


In [90]:
flat_amenities.to_csv('Data/flat_amenities.csv', index=False)

## Get dist from dhoby ghaut

In [91]:
flat_coord = pd.read_csv('Data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]

dist_dhoby = dist_from_location(flat_coord, (1.299308, 103.845285))
dist_dhoby = pd.DataFrame.from_dict(dist_dhoby).T
dist_dhoby = dist_dhoby.rename(columns={0: 'flat', 1: 'dist_dhoby'}).reset_index().drop(['index'], axis=1)
dist_dhoby.head()

,flat,dist_dhoby
0,684B EDGEDALE PLAINS,13.846865
1,630 PASIR RIS DR 3,13.789367
2,118 MARSILING RISE,17.000156
3,42 JLN BAHAGIA,3.341669
4,930 YISHUN CTRL 1,14.137783


In [92]:
flat_amenities = pd.read_csv('Data/flat_amenities.csv')
flat_amenities = flat_amenities.merge(dist_dhoby, on='flat', how='outer')
flat_amenities.head()

,flat,school,school_dist,num_school_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,dist_dhoby
0,684B EDGEDALE PLAINS,Horizon Primary School,0.423743,16,OASIS LRT STATION,0.268944,18,0.411221,12,13.846865
1,630 PASIR RIS DR 3,Park View Primary School,0.205457,6,PASIR RIS MRT STATION,1.084685,1,0.101067,6,13.789367
2,118 MARSILING RISE,Fuchun Primary School,0.901488,14,WOODLANDS MRT STATION,0.703757,5,0.805409,8,17.000156
3,42 JLN BAHAGIA,Hong Wen School,0.600572,14,BOON KENG MRT STATION,1.005326,8,0.283154,22,3.341669
4,930 YISHUN CTRL 1,Xishan Primary School,0.726366,14,YISHUN MRT STATION,0.382512,3,0.069918,15,14.137783


In [93]:
flat_amenities.to_csv('Data/flat_amenities.csv', index=False)